In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 13
WINTER = "2024-2025"
TITLE = "14-15 February 2025"
SUBTITLE = "8 AM 16 February 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 2, 15, 10)
ets = datetime(2025, 2, 17, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,AK,AFG,30.0,-148.240000,64.830000,POINT (-2107114.947 2920198.659),True,0,True,LSR,-78,115
1,CO,BOU,27.0,-106.600000,40.390000,POINT (-558681.271 -490150.462),True,1,True,LSR,-27,1
2,WY,RIW,25.0,-110.840000,42.820000,POINT (-881052.86 -183843.072),True,2,True,LSR,-38,11
3,WY,RIW,21.0,-110.860000,43.600000,POINT (-871331.151 -97460.76),True,3,True,LSR,-37,14
4,AK,AFG,20.0,-147.700000,64.820000,POINT (-2088859.029 2903689.134),True,4,True,LSR,-78,114
...,...,...,...,...,...,...,...,...,...,...,...,...
3311,IA,DVN,0.0,-91.740901,40.639403,POINT (695942.961 -449906.127),False,IA-VB-7,False,COCORAHS,14,3
3312,IA,DMX,0.0,-94.222148,43.066279,POINT (468943.311 -198417.384),False,IA-KS-4,False,COCORAHS,6,11
3313,IA,DVN,0.0,-91.805145,40.630803,POINT (690642.762 -451398.417),False,IA-VB-3,False,COCORAHS,14,3
3314,IA,DVN,0.0,-91.699706,41.993100,POINT (684742.262 -299642.014),False,IA-LN-49,False,COCORAHS,14,8


In [12]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'RKWI4',
        'SACI4',
        'IA-CN-17',
        1255,
        'IA-SH-8',
        756,
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('ESTI4', 0),
        ('IA-HM-4', 2.5),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=10)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250216.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
145     IA  OAX  7.0 -95.700000  41.740000   POINT (356715.962 -353111.65)   
225     IA  OAX  6.0 -96.020000  41.710000  POINT (330353.237 -357780.947)   
226     IA  OAX  6.0 -95.930000  41.710000  POINT (337816.029 -357415.083)   
227     IA  OAX  6.0 -95.900000  41.570000  POINT (341056.094 -372838.405)   
228     IA  OAX  6.0 -95.890000  41.550000  POINT (341994.732 -375017.843)   
...    ...  ...  ...        ...        ...                             ...   
3311    IA  DVN  0.0 -91.740901  40.639403  POINT (695942.961 -449906.127)   
3312    IA  DMX  0.0 -94.222148  43.066279  POINT (468943.311 -198417.384)   
3313    IA  DVN  0.0 -91.805145  40.630803  POINT (690642.762 -451398.417)   
3314    IA  DVN  0.0 -91.699706  41.993100  POINT (684742.262 -299642.014)   
3315    IA  DVN  0.0 -91.580730  41.870510  POINT (695872.692 -312217.468)   

      used_for_analysis     nwsli  plotme    source  xcell  yce

## 